In [8]:
from scipy import stats as st
import pandas as pd

In [9]:
csv_path = "../dataset/preprocessed.csv"

In [10]:
data = pd.read_csv(csv_path)
data.describe()

,Age,Number of Cigarettes Beedi per day,Number of years smoked,Height,Weight,Heart_Rate,BP_Systolic,BP_Diastolic,Exact Time of Symptom Onset,BASELINE SERUM CREATININE,...,Liporotein A,Admission Time,Age 65-74/>75,Systolic BP<100,Heart Rate>100,KILLIP II-IV,Anterior STE or LBBB,"H/O Diabetes, HTN or Angina",Weight < 67 Kgs,Time to Treatment >4 Hours
count,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,...,1630.000000,1630.00000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000
mean,54.338650,5.242945,8.138650,165.187485,64.483933,84.338037,122.509816,77.006748,700.613497,1.053699,...,46.839387,766.83681,0.480982,0.373620,0.250307,0.343558,0.563804,0.450307,0.582822,0.611043
std,11.923561,7.367816,11.352618,6.682856,10.523991,18.654277,24.337021,14.015841,395.479670,0.435791,...,58.774855,401.75035,0.943105,0.990893,0.661989,0.754608,0.496065,0.497677,0.493244,0.487663
min,18.000000,0.000000,0.000000,136.000000,38.000000,34.000000,40.000000,20.000000,0.000000,0.100000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,46.000000,0.000000,0.000000,161.115000,57.000000,75.000000,110.000000,70.000000,360.000000,0.800000,...,0.000000,480.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,55.000000,0.000000,0.000000,165.780000,64.000000,82.000000,120.000000,80.000000,665.000000,1.000000,...,20.450000,780.00000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
75%,62.000000,10.000000,15.000000,169.650000,72.000000,94.000000,138.000000,90.000000,1050.000000,1.200000,...,81.300000,1109.00000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,96.000000,50.000000,45.000000,182.880000,112.000000,191.000000,220.000000,140.000000,1435.000000,5.900000,...,248.300000,1436.00000,3.000000,3.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
num = ['Age', 'Number of Cigarettes Beedi per day', 'Number of years smoked', 'Height', 'Weight', 'Heart_Rate', 'BP_Systolic', 'BP_Diastolic',
        'Exact Time of Symptom Onset', 'BASELINE SERUM CREATININE', 'HAEMOGLOBIN', 'HBA1C', 'Total_cholesterol', 'LDL', 'HDL', 'Triglycerides',
        'APO B(g/l)', 'Liporotein A', 'Admission Time'
      ]

In [12]:
for col in data.columns:
    data[col] = data[col].fillna(0)

In [13]:
target = "Follow Up Status"

In [14]:
labels = data[target].unique()
labels

array(['Alive', 'Deceased'], dtype=object)

In [15]:
test_stats = {}
p_vals = {}

alpha = 0.05

In [16]:
for col in data.columns:
    if(data[col].unique().shape[0]==1):
        continue

    if (col != target):
        
        counts = [data[data[target] == label][col] for label in labels]

        if col not in num:
            observed = pd.crosstab(index = data[col], columns = data[target])
            test_stat, p_value, _, _ = st.chi2_contingency(observed)
        else:
            test_stat, p_value = st.ttest_ind(a=counts[0],b=counts[1],equal_var=False)

        test_stats[col] = test_stat
        p_vals[col] = p_value

In [17]:
results = pd.DataFrame({'Feature' : list(p_vals.keys()),
                        'CV' : list(test_stats.values()),
                        'p value' : list(p_vals.values()),
                        'Significant' : ['S' if p_val < alpha else 'X' for p_val in p_vals.values()]
                        })

results_xl_path = "feature_tests.xlsx"

results.to_excel(results_xl_path, index = False)